In [5]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse

# def scrape_section_page(section_url, class_names):
#     response = requests.get(section_url)
#     soup = BeautifulSoup(response.content, 'html.parser')
    
#     section_posts = []
#     for class_name in class_names:
#         if class_name == "none":
#             continue
#         news_list_section = soup.find_all('div', class_=class_name)
#         for news_div in news_list_section:
#             news_links = news_div.find_all('a', href=True, title=True)
#             for link in news_links:
#                 post_title = link['title'].strip()
#                 post_url = urljoin(section_url, link['href'])
#                 if is_valid_url(post_url):
#                     section_posts.append({'title': post_title, 'url': post_url})
#                 else:
#                     print(f"Invalid URL found: {post_url}")

#     return section_posts

# def scrape_section_page(section_url, class_names):
#     response = requests.get(section_url)
#     soup = BeautifulSoup(response.content, 'html.parser')

#     section_posts = []
#     for class_name in class_names:
#         if class_name == "none":
#             continue
#         news_list_section = soup.find_all('div', class_=class_name)
#         for news_div in news_list_section:
#             news_links = news_div.find_all('a', href=True, title=True)
#             for link in news_links:
#                 post_title = link['title'].strip()
#                 post_url = link['href']
#                 if is_valid_url(post_url, section_url):
#                     section_posts.append({'title': post_title, 'url': urljoin(section_url, post_url)})
#                 else:
#                     print(f"Invalid URL found: {urljoin(section_url, post_url)}")

#     return section_posts

# def scrape_section_page(section_url, class_names):
#     response = requests.get(section_url)
#     soup = BeautifulSoup(response.content, 'html.parser')
    
#     section_posts = []
#     for class_name in class_names:
#         if class_name == "none":
#             continue
#         news_list_section = soup.find_all('div', class_=class_name)
#         for news_div in news_list_section:
#             news_links = news_div.find_all('a', href=True, title=True)
#             for link in news_links:
#                 post_title = link['title'].strip()
#                 post_url = link['href']
#                 if is_valid_url(post_url, section_url):
#                     section_posts.append({'title': post_title, 'url': urljoin(section_url, post_url)})
#                 else:
#                     print(f"Invalid URL found: {urljoin(section_url, post_url)}")

#     return section_posts

def scrape_section_page(section_url, class_names):
    response = requests.get(section_url)
    soup = BeautifulSoup(response.content, 'html.parser')

    section_posts = []
    for class_name in class_names:
        if class_name == "none":
            continue
        news_list_section = soup.find_all('div', class_=class_name)
        for news_div in news_list_section:
            news_links = news_div.find_all('a', href=True)
            for link in news_links:
                link_url = urljoin(section_url, link.get('href', ''))
                if is_valid_url(link_url, section_url):
                    news_links_text = link.get_text().strip()
                    section_posts.append({'url': link_url, 'title': news_links_text})

    return section_posts

def is_valid_url(url, base_url):
    parsed = urlparse(urljoin(base_url, url))
    return all([parsed.scheme, parsed.path, parsed.netloc])

def scrape_post_content(post_url):
    response = requests.get(post_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Remove unwanted elements
    for auth_div in soup.find_all('div', class_='auth-name-dt'):
        auth_div.decompose()
    for author_div in soup.find_all('div', class_='h-author'):
        author_div.decompose()
    for author_div in soup.find_all('div', class_='col-md-4'):
        author_div.decompose()
    for author_div in soup.find_all('div', class_='d-mags'):
        author_div.decompose()
    for author_div in soup.find_all('div', class_='d-social'):
        author_div.decompose()
    for author_div in soup.find_all('div', class_='d-nav-item-info'):
        author_div.decompose()

    # Extract content
    paragraphs = soup.find_all('p')
    content = "\n\n".join([p.text.strip() for p in paragraphs])
    
    return content

def is_valid_url(url):
    parsed = urlparse(url)
    return all([parsed.scheme, parsed.netloc])

def main(language_urls):
    all_results = {}

    for lang, urls in language_urls.items():
        print(f"Scraping data for language: {lang}")
        
        base_url = urls["base"]
        weather_class = urls["weather_class"]
        khati_class = urls["khati_badi_class"]
        main_class = urls["main_class"]
        news_classes = urls["news_classes"]
        government_classes = urls["government_class"]
        
        # Scrape the main page
        main_posts = scrape_section_page(base_url, main_class)
        khabar_posts = scrape_section_page(base_url, news_classes)

        # Scrape the weather page if available
        weather_posts = scrape_section_page(base_url, weather_class)

        # Scrape the Khati Badi page
        khati_posts = scrape_section_page(base_url, khati_class)

        government_post = scrape_section_page(base_url, government_classes)

        # Set to track processed URLs
        processed_urls = set()

        # Dictionary to store the results for this language
        results = {
            "Main Page Posts": [],
            "Khabar Section Posts": [],
            "Weather Posts": [],
            "Khati Badi Posts": [],
            "government posts":[]
        }

        # Process main page posts
        for post in main_posts:
            if post['url'] not in processed_urls:
                content = scrape_post_content(post['url'])
                results["Main Page Posts"].append({
                    "title": post['title'],
                    "link": post['url'],
                    "content": content
                })
                processed_urls.add(post['url'])     

        # Process "Khabar" section posts
        for post in khabar_posts:
            if post['url'] not in processed_urls:
                content = scrape_post_content(post['url'])
                results["Khabar Section Posts"].append({
                    "title": post['title'],
                    "link": post['url'],
                    "content": content
                })
                processed_urls.add(post['url'])

        # Process weather posts
        for post in weather_posts:
            if post['url'] not in processed_urls:
                content = scrape_post_content(post['url'])
                results["Weather Posts"].append({
                    "title": post['title'],
                    "link": post['url'],
                    "content": content
                })
                processed_urls.add(post['url'])

        # Process Khati Badi posts
        for post in khati_posts:
            if post['url'] not in processed_urls:
                content = scrape_post_content(post['url'])
                results["Khati Badi Posts"].append({
                    "title": post['title'],
                    "link": post['url'],
                    "content": content
                })
                processed_urls.add(post['url'])
       
        for post in government_post:
            if "asomiya" in language_urls or "odia" in language_urls:
                if post['url'] not in processed_urls:
                    if "https://telugu.krishijagran.com/government-schemes" in post['url']:
                        content = scrape_post_content(post['url'])
                        results["government posts"].append({
                            "title": post['title'],
                            "link": post['url'],
                            "content": content
                        })
                        processed_urls.add(post['url'])

            if "asomiya" in language_urls or "odia" in language_urls:
                if post['url'] not in processed_urls:
                    if "https://asomiya.krishijagran.com/government-schemes" in post['url'] or "https://odia.krishijagran.com/government-schemes/" in post['url']:
                        content = scrape_post_content(post['url'])
                        results["government posts"].append({
                            "title": post['title'],
                            "link": post['url'],
                            "content": content
                        })
                        processed_urls.add(post['url'])
            else:
                if post['url'] not in processed_urls:
                    content = scrape_post_content(post['url'])
                    results["government posts"].append({
                        "title": post['title'],
                        "link": post['url'],
                        "content": content
                    })
                    processed_urls.add(post['url'])

        all_results[lang] = results

    return all_results

# Define the base URLs and class names for each language
language_urls = {
     # "hindi": {
    #     "base": "https://hindi.krishijagran.com/",
    #     "main_class": ["home-top-l"],
    #     "news_classes": ["home-top-news-lst"],
    #     "weather_class": ["weather-home mt-5 mb-3"],
    #     "khati_badi_class": ["home-2-3-lst"],
    # },
    # "english": {
    #     "base": "https://krishijagran.com/",
    #     "main_class": ["home-top-l"],
    #     "news_classes": ["home-top-news-lst"],
    #     "weather_class": ["none"],
    #     "khati_badi_class": ["home-2-3-lst"],
    # },
    # "punjabi": {
    #     "base": "https://punjabi.krishijagran.com/",
    #     "main_class": ["none"],
    #     "news_classes": ["home-top-news-lst"],
    #     "weather_class": ["home-top-l"],
    #     "khati_badi_class": ["home-2-3-lst"],
    # },
    # "marathi": {
    #     "base": "https://marathi.krishijagran.com/",
    #     "main_class": ["home-top-l"],
    #     "news_classes": ["home-top-news-lst"],
    #     "weather_class": ["weather-home mt-5 mb-3"],
    #     "khati_badi_class": ["none"],
    # },
    # "tamil": {
    #     "base": "https://tamil.krishijagran.com/",
    #     "main_class": ["home-top-l"],
    #     "news_classes": ["home-top-news-lst"],
    #     "weather_class": ["none"],
    #     "khati_badi_class": ["home-2-3-lst"],
    # },
    # "malayam": {
    #     "base": "https://malayalam.krishijagran.com/",
    #     "main_class": ["home-top-l"],
    #     "news_classes": ["home-top-news-lst"],
    #     "weather_class": ["none"],
    #     "khati_badi_class": ["none"],
    # },
    #   "bengali": {
    #     "base": "https://bengali.krishijagran.com/",
    #     "main_class": ["home-top-l"],
    #     "news_classes": ["home-top-news-lst"],
    #     "weather_class": ["weather-home mt-5 mb-3"],
    #     "khati_badi_class": ["none"],
    #     "government_class":["h-cat-lst shadow-sm"]
    # },
    #   "kannada": {
    #     "base": "https://kannada.krishijagran.com/",
    #     "main_class": ["home-top-l"],
    #     "news_classes": ["home-top-news-lst"],
    #     "weather_class": ["none"],
    #     "khati_badi_class": ["none"],
    #     "government_class":["h-cat-lst shadow-sm"]
    # },
    #   "odia": {
    #     "base": "https://odia.krishijagran.com/",
    #     "main_class": ["home-top-l"],
    #     "news_classes": ["home-top-news-lst"],
    #     "weather_class": ["none"],
    #     "khati_badi_class": ["three-boxes"],
    #     "government_class":["h-cat-lst shadow-sm"]
    # },
    #   "asomiya": {
    #     "base": "https://asomiya.krishijagran.com/",
    #     "main_class": ["home-top-l"],
    #     "news_classes": ["home-top-news-lst"],
    #     "weather_class": ["none"],
    #     "khati_badi_class": ["none"],
    #     "government_class":["h-cat-lst shadow-sm"],
    #     # "government_class":["list-unstyled"]
    # },
    "gujarati": {
         "base": "https://gujarati.krishijagran.com/",
         "main_class": ["none"],
         "news_classes": ["col-xs-12 col-sm-6 col-md-6 col-lg-6 cat-flex"],
         "weather_class": ["none"],
         "khati_badi_class": ["none"],
         "government_class":["col-xs-12 col-sm-6 col-md-6 col-lg-6 cat-flex"]
    },
    #   "telugu": {
    #     "base": "https://telugu.krishijagran.com/",
    #     "main_class": ["row e-h"],
    #     "news_classes": ["h-cat-lst shadow-sm"],
    #     "weather_class": ["none"],
    #     "khati_badi_class": ["none"],
    #     "government_class":["col-xs-12 col-sm-6 col-md-6 col-lg-6 cat-flex"],
        # "government_class":["list-unstyled"]
    # },
}

# Start the scraping process and get the results
scraped_data = main(language_urls)
print(scraped_data)


Scraping data for language: gujarati


TypeError: is_valid_url() takes 1 positional argument but 2 were given